In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer
from scipy.sparse import csr_matrix, hstack

In [3]:
!gsutil ls gs://smartandnimble/identifyme

gs://smartandnimble/identifyme/
gs://smartandnimble/identifyme/test_sparse_w5.pkl
gs://smartandnimble/identifyme/train_part_sparse_w5.pkl
gs://smartandnimble/identifyme/train_sparse_w5.pkl
gs://smartandnimble/identifyme/valid_sparse_w5.pkl
gs://smartandnimble/identifyme/y_train_w5.csv
gs://smartandnimble/identifyme/y_valid_w5.csv
gs://smartandnimble/identifyme/y_w5.csv


In [4]:
!gsutil cp -r gs://smartandnimble/identifyme/xgb "data/identifyme/"

Copying gs://smartandnimble/identifyme/xgb/test.buffer...
/ [1 files][  4.9 MiB/  4.9 MiB]                                                Copying gs://smartandnimble/identifyme/xgb/train.buffer...
Copying gs://smartandnimble/identifyme/xgb/trainfull.buffer...                  / [2 files][ 14.3 MiB/ 14.3 MiB]                                                
/ [2 files][ 14.3 MiB/ 32.3 MiB]                                                -- [3 files][ 32.3 MiB/ 32.3 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://smartandnimble/identifyme/xgb/valid.buffer...
- [3 files][ 32.3 MiB/ 36.3 MiB]                                                - [4 files][ 36.3 MiB/ 36.3 MiB]                                                
Operatio

In [2]:
#This is testing on w5_old
folder = "data/identifyme/xgb/"

dtrain = xgb.DMatrix(folder+"train.buffer")
dvalid = xgb.DMatrix(folder+"valid.buffer")
dtrainfull = xgb.DMatrix(folder+"trainfull.buffer")
dtest = xgb.DMatrix(folder+"test.buffer")

y_valid=pd.read_csv(folder+"y_valid_w5_old.csv", header=None, squeeze=True)

In [ ]:
%%time
# specify parameters via map
param = {'colsample_bytree': 0.5, 'silent': 0, 'eval_metric': 'merror', \
 'max_delta_step': 7, 'nthread': 3, 'min_child_weight': 1, \
 'subsample': 0.9500000000000001, \
 'eta': 0.17500000000000002, 'objective': 'multi:softmax', \
 'num_class': 550, 'max_depth': 13, 'gamma': 0.8500000000000001}
#param = {'silent':0, 'objective':'multi:softmax', 'num_class':550, 'max_depth':10, 'max_delta_step': 3}
#param['eval_metric'] = ['merror']#, 'mlogloss']
#param['nthread'] = 1
param['model_dir'] = folder + "model"
param['model_out'] = folder + "model/final_model_w5_old.xgb.model"
param['save_period'] = 100
num_round = 790
#evallist  = [(dvalid,'eval'), (dtrain,'train')]
bst = xgb.train(param, dtrain, num_round) #, evallist, verbose_eval=True, early_stopping_rounds=10)
# make prediction
preds = bst.predict(dvalid)
print "Accuracy:", accuracy_score(preds, y_valid)

In [4]:
preds

array([ 319.,  529.,   17., ...,  398.,  205.,  357.], dtype=float32)

In [5]:
print "Accuracy:", accuracy_score(preds, y_valid)

Accuracy: 0.53582484801


In [6]:
bst.save_model(folder+'xgb_w5_old.model')

In [3]:
bst = xgb.Booster({'nthread':4}) #init model
bst.load_model(folder+'xgb_w5_old.model') # load data


In [4]:
results = bst.predict(dtest)

In [5]:
results

array([ 354.,  516.,  107., ...,  450.,  409.,  409.], dtype=float32)

In [7]:
class_encoder = LabelEncoder().fit(y_valid.astype('str'))

In [9]:
xgb_results = class_encoder.inverse_transform(results.astype(int))

In [10]:
xgb_results

array(['417', '69', '195', ..., '503', '467', '467'], dtype=object)